## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = ["Age", "DistanceFromHome", "Education", "HourlyRate", "JobRole", "JobSatisfaction", "NumCompaniesWorked", "WorkLifeBalance", "YearsAtCompany", "YearsSinceLastPromotion"]


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Age                         int64
DistanceFromHome            int64
Education                   int64
HourlyRate                  int64
JobRole                    object
JobSatisfaction             int64
NumCompaniesWorked          int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
dtype: object

In [63]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [64]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
from sklearn.preprocessing import OneHotEncoder
def ohe_X(column, X_train, X_test):
    encoder = OneHotEncoder(sparse_output=False)
    encoder.fit(X_train[[column]])
    encoded_train = encoder.transform(X_train[[column]])
    encoded_test = encoder.transform(X_test[[column]])
    encoded_columns = encoder.get_feature_names_out([column])
    encoded_train_df = pd.DataFrame(encoded_train, columns=encoded_columns)
    encoded_test_df = pd.DataFrame(encoded_test, columns=encoded_columns)
    return encoded_train_df, encoded_test_df


In [65]:
def process_df(column, encoded_train, encoded_test, X_train, X_test):
    X_train_sorted = X_train.reset_index()
    X_test_sorted = X_test.reset_index()
    X_processed_train = pd.concat([X_train_sorted, encoded_train], axis=1)
    X_processed_train = X_processed_train.drop(['index', column], axis=1)
    X_processed_test = pd.concat([X_test_sorted, encoded_test], axis=1)
    X_processed_test = X_processed_test.drop(['index', column], axis=1)
    return X_processed_train, X_processed_test

In [66]:
JobRole_encoded_train, JobRole_encoded_test = ohe_X('JobRole', X_train, X_test)
JobRole_encoded_train

,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1097,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1098,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1099,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1100,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [69]:
JobRole_encoded_test

,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
363,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
364,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
365,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
366,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
X_processed_train, X_processed_test = process_df("JobRole", JobRole_encoded_train, JobRole_encoded_test, X_train, X_test)
X_processed_train

,Age,DistanceFromHome,Education,HourlyRate,JobSatisfaction,NumCompaniesWorked,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,37,1,3,59,4,4,4,6,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,27,3,4,54,4,0,3,4,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,30,12,4,74,1,1,4,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,36,7,4,65,4,2,3,6,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,56,1,2,90,1,2,4,3,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,33,1,4,84,2,1,3,6,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1098,29,1,3,87,4,3,2,7,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1099,40,6,2,38,4,1,3,21,7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1100,43,16,3,80,4,5,3,17,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [68]:
X_processed_test

,Age,DistanceFromHome,Education,HourlyRate,JobSatisfaction,NumCompaniesWorked,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,37,10,4,58,1,1,1,10,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,18,3,2,70,4,1,4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,30,22,3,48,3,4,3,5,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,34,10,4,87,3,1,3,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,56,23,3,68,2,4,3,19,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,37,7,4,30,3,4,4,5,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
364,45,28,3,97,4,4,3,5,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
365,31,1,2,98,1,1,3,4,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
366,31,15,3,72,3,1,2,2,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_processed_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_processed_train)
X_test_scaled = scaler.transform(X_processed_test)
X_train_scaled


array([[-5.97353422e-04, -1.00933748e+00,  8.08431209e-02, ...,
        -4.96594739e-01, -5.31871298e-01, -2.39961609e-01],
       [-1.09773647e+00, -7.61501047e-01,  1.07072222e+00, ...,
         2.01371445e+00, -5.31871298e-01, -2.39961609e-01],
       [-7.68594737e-01,  3.53762891e-01,  1.07072222e+00, ...,
        -4.96594739e-01, -5.31871298e-01,  4.16733328e+00],
       ...,
       [ 3.28544382e-01, -3.89746401e-01, -9.09035982e-01, ...,
        -4.96594739e-01, -5.31871298e-01, -2.39961609e-01],
       [ 6.57686118e-01,  8.49435752e-01,  8.08431209e-02, ...,
        -4.96594739e-01, -5.31871298e-01, -2.39961609e-01],
       [-1.42687821e+00,  4.77681106e-01, -9.09035982e-01, ...,
        -4.96594739e-01, -5.31871298e-01,  4.16733328e+00]])

In [73]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train = department_encoder.transform(y_train[['Department']])
y_department_test = department_encoder.transform(y_test[['Department']])
y_department_train

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [75]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train = attrition_encoder.transform(y_train[['Attrition']])
y_attrition_test = attrition_encoder.transform(y_test[['Attrition']])
y_attrition_train

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [105]:
# Find the number of columns in the X training data
num_columns = len(X_processed_train.columns)

# Create the input layer
input_layer = layers.Input(num_columns, name='input')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)

In [106]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(len(y_df['Department'].unique()), activation='softmax', name='department_output')(department_hidden_layer)


In [107]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(len(y_df['Attrition'].unique()), activation='sigmoid', name='attrition_output')(attrition_hidden_layer)


In [108]:
# Create the model
model = Model(
    inputs=input_layer,
    outputs=[
        department_output_layer,
        attrition_output_layer
    ]
)

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output':'categorical_crossentropy',
        'attrition_output':'binary_crossentropy'
    },
    metrics={
        'department_output':'accuracy',
        'attrition_output':'accuracy'
    }
)

# Summarize the model
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 18)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   1216      ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                            

In [109]:
# Train the model
model.fit(
    X_train_scaled,
    {
        'department_output':y_department_train,
        'attrition_output':y_attrition_train
    },
    epochs=100,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/100
28/28 [==============================] - 2s 20ms/step - loss: 1.2132 - department_output_loss: 0.7371 - attrition_output_loss: 0.4760 - department_output_accuracy: 0.6663 - attrition_output_accuracy: 0.8320 - val_loss: 0.9729 - val_department_output_loss: 0.4662 - val_attrition_output_loss: 0.5067 - val_department_output_accuracy: 0.8914 - val_attrition_output_accuracy: 0.8281
Epoch 2/100
28/28 [==============================] - 0s 7ms/step - loss: 0.6719 - department_output_loss: 0.2688 - attrition_output_loss: 0.4031 - department_output_accuracy: 0.9319 - attrition_output_accuracy: 0.8513 - val_loss: 0.6450 - val_department_output_loss: 0.1711 - val_attrition_output_loss: 0.4739 - val_department_output_accuracy: 0.9593 - val_attrition_output_accuracy: 0.8281
Epoch 3/100
28/28 [==============================] - 0s 6ms/step - loss: 0.4844 - department_output_loss: 0.1107 - attrition_output_loss: 0.3737 - department_output_accuracy: 0.9625 - attrition_output_accuracy: 0.8513

In [111]:
# Evaluate the model with the testing data
results = model.evaluate(
    np.array(X_test_scaled),
     {
        'department_output':y_department_test,
        'attrition_output':y_attrition_test
     }
  )

12/12 [==============================] - 0s 4ms/step - loss: 2.3997 - department_output_loss: 0.3449 - attrition_output_loss: 2.0548 - department_output_accuracy: 0.9457 - attrition_output_accuracy: 0.8125


In [114]:
# Print the accuracy for both department and attrition
print(f'Attrition predictions accuracy: {results[3]}')
print(f'Department predictions accuracy: {results[4]}')

Attrition predictions accuracy: 0.945652186870575
Department predictions accuracy: 0.8125


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.